# Extraction: Extract structured data from text and other unstructured media using chat models and few-shot examples.

Trong hướng dẫn này, chúng ta sẽ sử dụng các tính năng gọi công cụ (tool calling) của mô hình trò chuyện để trích xuất thông tin có cấu trúc từ văn bản không có cấu trúc. Chúng tôi cũng sẽ trình bày cách sử dụng nhắc nhở (prompt) ít lần trong bối cảnh này để cải thiện hiệu suất.

## Content
1. Setup
2. LangSmith
3. The Schema
4. The Extractor
5. Multiple Entities
6. Reference examples

# 1. Setup

In [ ]:
#pip install --upgrade langchain-core

# 2. LangSmith

In [ ]:
import os
import getpass

os.environ["LANGSMITH_TRAING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass()

In [ ]:
# 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

# 3. The Schema

Đầu tiên, chúng ta cần mô tả thông tin nào chúng ta muốn trích xuất từ ​​văn bản. Chúng ta sẽ sử dụng Pydantic để xác định một lược đồ ví dụ để trích xuất thông tin cá nhân.

In [4]:
from typing import Optional
from pydantic import BaseModel
from pydantic import Field

class Person(BaseModel):
    """Information about a person"""
    # Doc-string for the entity person
    # This doc-string is sent to LLM as the description of the schema person,
    # and it can help to improve axtraction results.

    # Note that:
    # 1. Each field as an `optional` -- this allows the model to decline to extract it
    # 2. Each field has a `description` -- this description is used by LLM
    # Having a good description can help improve extraction result

    name: Optional[str] = Field(default=None, description="The name of the person")
    hair_color: Optional[str] = Field(default=None, description="The color of the person's hair if know")
    height_in_meters: Optional[str] = Field(default=None, description="Height measured in meters")

Có hai cách thực hành tốt nhất khi xác định lược đồ: 

    1. Ghi lại các thuộc tính và chính lược đồ: Thông tin này được gửi đến LLM và được sử dụng để cải thiện chất lượng trích xuất thông tin.
    2. Đừng ép LLM phải tạo thông tin! Ở trên, chúng tôi đã sử dụng Optional cho các thuộc tính cho phép LLM đưa ra None nếu không biết câu trả lời.

Để có hiệu suất tốt nhất, hãy ghi lại lược đồ thật kỹ và đảm bảo mô hình không bị ép phải trả về kết quả nếu không có thông tin nào cần trích xuất trong văn bản.

# 4. The Extractor

Hãy tạo một trình trích xuất thông tin bằng cách sử dụng lược đồ đã xác định ở trên.

In [1]:
from typing import Optional
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import MessagesPlaceholder
from pydantic import BaseModel
from pydantic import Field

# Define a custom prompt to provide instructions and any additional context.
# 1) You can add examples into the prompt template to improve extraction quality
# 2) Introduce additional parameters to take context into account (e.g., include metadata
#    about the document from which the text was extracted.)

prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an expert extraction algorithm."
            "Only extract relevant information from the text."
            "If you do not know the value of an attribute asked to extract,"
            "return null for the attribute's value.",
        ),
        ("human", "{text}")
    ]
)

Chúng ta cần sử dụng một mô hình hỗ trợ việc gọi hàm/công cụ.

In [5]:
import os
import getpass
from langchain.chat_models import init_chat_model

if not os.environ.get("GROQ_API_KEY"):
    os.environ["GROQ_API_KEY"] = getpass.getpass("Enter API key for Groq: ")

llm = init_chat_model("llama3-8b-8192", model_provider="groq")

In [6]:
structured_llm = llm.with_structured_output(schema=Person)

In [7]:
text = "Alan Smith is 6 feet tall and has blond hair."
prompt = prompt_template.invoke({"text": text})
structured_llm.invoke(prompt)

Person(name='Alan Smith', hair_color='blond', height_in_meters='1.83')

# 5. Multiple Entities

Trong hầu hết các trường hợp, bạn nên trích xuất danh sách các thực thể thay vì một thực thể duy nhất. Điều này có thể dễ dàng đạt được bằng cách sử dụng pydantic bằng cách lồng các mô hình vào nhau.

In [ ]:
from typing import List, Optional
from pydantic import BaseModel, Field


class Person(BaseModel):
    """Information about a person."""

    # ^ Doc-string for the entity Person.
    # This doc-string is sent to the LLM as the description of the schema Person,
    # and it can help to improve extraction results.

    # Note that:
    # 1. Each field is an `optional` -- this allows the model to decline to extract it!
    # 2. Each field has a `description` -- this description is used by the LLM.
    # Having a good description can help improve extraction results.
    name: Optional[str] = Field(default=None, description="The name of the person")
    hair_color: Optional[str] = Field(default=None, description="The color of the person's hair if known")
    height_in_meters: Optional[str] = Field(default=None, description="Height measured in meters")


class Data(BaseModel):
    """Extracted data about people."""
    # Creates a model so that we can extract multiple entities.
    people: List[Person]

    IMPORTANT
Kết quả trích xuất có thể không hoàn hảo ở đây. Đọc tiếp để xem cách sử dụng Ví dụ tham khảo để cải thiện chất lượng trích xuất và xem hướng dẫn cách trích xuất của chúng tôi để biết thêm chi tiết.

    https://python.langchain.com/docs/how_to/#extraction

In [ ]:
structured_llm = llm.with_structured_output(schema=Data)
text = "My name is Jeff, my hair is black and i am 6 feet tall. Anna has the same color hair as me."
prompt = prompt_template.invoke({"text": text})
structured_llm.invoke(prompt)

    Tip

Khi lược đồ hỗ trợ việc trích xuất nhiều thực thể, nó cũng cho phép mô hình không trích xuất bất kỳ thực thể nào nếu không có thông tin liên quan nào trong văn bản bằng cách cung cấp một danh sách trống. Thông thường, đây là một điều tốt! Nó cho phép chỉ định các thuộc tính bắt buộc trên một thực thể mà không nhất thiết buộc mô hình phải phát hiện thực thể này.

# 6. Reference examples

Hành vi của các ứng dụng LLM có thể được điều khiển bằng cách sử dụng lời nhắc ít lần. Đối với các mô hình trò chuyện, điều này có thể diễn ra dưới dạng một chuỗi các cặp thông báo đầu vào và phản hồi thể hiện các hành vi mong muốn. Ví dụ, chúng ta có thể truyền đạt ý nghĩa của một biểu tượng bằng cách xen kẽ các thông báo của người dùng và trợ lý:

In [17]:
messages = [
    {"role": "user", "content": "2 🦜 2"},
    {"role": "assistant", "content": "4"},
    {"role": "user", "content": "2 🦜 3"},
    {"role": "assistant", "content": "5"},
    {"role": "user", "content": "3 🦜 4"},
]

response = llm.invoke(messages)
print(response.content)

7


Đầu ra có cấu trúc thường sử dụng lệnh gọi công cụ ẩn. Điều này thường liên quan đến việc tạo ra các thông báo AI chứa lệnh gọi công cụ, cũng như các thông báo công cụ chứa kết quả của lệnh gọi công cụ. Một chuỗi thông báo trông như thế nào trong trường hợp này? Các nhà cung cấp mô hình trò chuyện khác nhau áp đặt các yêu cầu khác nhau đối với chuỗi thông báo hợp lệ. Một số sẽ chấp nhận chuỗi thông báo (lặp lại) có dạng: Thông báo người dùng Thông báo AI có lệnh gọi công cụ Thông báo công cụ có kết quả

Hãy thử điều này. Chúng ta có thể chuyển đổi các cặp chuỗi đầu vào và các đối tượng Pydantic mong muốn thành một chuỗi các tin nhắn có thể được cung cấp cho một mô hình trò chuyện. Về cơ bản, LangChain sẽ định dạng các lệnh gọi công cụ theo định dạng bắt buộc của từng nhà cung cấp.


In [ ]:
from langchain_core.utils.function_calling import tool_example_to_messages

examples = [
    ("The ocean is vast and blue. It's more than 20,000 feet deep.", Data(people=[])),
    ( "Fiona traveled far from France to Spain.", Data(people=[Person(name='Fiona', height_in_meters=None, hair_color=None)])),
]

messages = []

for txt, tool_call in examples:
    if tool_call.people:
        # This final message is optional for some providers
        ai_response = "Detected people."
    else:
        ai_response = "Detected no people"
    messages.extend(tool_example_to_messages(txt, [tool_call], ai_response=ai_response))
    # Hàm tool_example_to_messages chuyển đổi (các) ví dụ (example) thành (các) tin nhắn (message)
    # để có thể tương thích khi đưa vào mô hình LLM
    """
    Example:
        HumanMessage: "The ocean is vast and blue. It's more than 20,000 feet deep."
        AIMessage: { "name": None, "height_in_meters": None, "hair_color": None }
        ToolMessage: "Tool call executed successfully"
        AIMessage (optional): "Detected no people."
    """
    # Hàm này đặc biệt hữu ích khi làm việc với LLM có hỗ trợ tool calling và cần phản hồi có cấu trúc.

C:\Users\NGOVANUC\AppData\Local\Temp\ipykernel_23780\4046823788.py:16: LangChainBetaWarning: The function `tool_example_to_messages` is in beta. It is actively being worked on, so the API may change.
  messages.extend(tool_example_to_messages(txt, [tool_call], ai_response=ai_response))


In [20]:
for message in messages:
    message.pretty_print()

================================ Human Message =================================

The ocean is vast and blue. It's more than 20,000 feet deep.
================================== Ai Message ==================================
Tool Calls:
  Data (2bd5a52a-fb43-4f1e-b252-9ea596684d85)
 Call ID: 2bd5a52a-fb43-4f1e-b252-9ea596684d85
  Args:
    people: []
================================= Tool Message =================================

You have correctly called this tool.
================================== Ai Message ==================================

Detected no people
================================ Human Message =================================

Fiona traveled far from France to Spain.
================================== Ai Message ==================================
Tool Calls:
  Data (545455fe-78a0-49ae-b525-987e1228e804)
 Call ID: 545455fe-78a0-49ae-b525-987e1228e804
  Args:
    people: [{'name': 'Fiona', 'hair_color': None, 'height_in_meters': None}]
=================================

Hãy so sánh hiệu suất có và không có các thông báo này. Ví dụ, hãy truyền một thông báo mà chúng ta không muốn trích xuất bất kỳ người nào:


In [23]:
message_no_extraction = {
    "role": "user",
    "content": "The solar system is large, but earth has only 1 moon.",
}

structured_llm = llm.with_structured_output(schema=Data)
structured_llm.invoke([message_no_extraction])

Data(people=[Person(name='Astronomy Enthusiast', hair_color=None, height_in_meters=None)])

Trong ví dụ này, mô hình có thể tạo ra các bản ghi sai về mọi người. Vì các ví dụ ít cảnh của chúng tôi chứa các ví dụ về "phủ định", chúng tôi khuyến khích mô hình hoạt động chính xác trong trường hợp này:

In [22]:
structured_llm.invoke(messages + [message_no_extraction])

Data(people=[])

Xem hướng dẫn này để biết thêm chi tiết về quy trình trích xuất với các ví dụ tham khảo, bao gồm cách kết hợp các mẫu nhắc nhở và tùy chỉnh việc tạo các thông báo ví dụ.

    https://python.langchain.com/docs/how_to/extraction_examples/


# 7. Next steps

Bây giờ bạn đã hiểu những điều cơ bản về trích xuất bằng LangChain, bạn đã sẵn sàng để tiến hành các hướng dẫn còn lại: 
- Thêm ví dụ: Chi tiết hơn về việc sử dụng các ví dụ tham khảo để cải thiện hiệu suất.
- Xử lý văn bản dài: Bạn nên làm gì nếu văn bản không phù hợp với cửa sổ ngữ cảnh của LLM?
- Sử dụng phương pháp phân tích cú pháp: Sử dụng phương pháp dựa trên lời nhắc để trích xuất bằng các mô hình không hỗ trợ công cụ/chức năng
